In [2]:
import sys
sys.path.append('../script/')
sys.path.append('../class/')

import pandas as pd
import numpy as np
import sunnytools as st

%load_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
sns.set_style('darkgrid')
pd.set_option('display.max_rows', 500)
plt.rcParams['figure.figsize'] = (20.0, 5.0)

# prepare some data
df = pd.read_csv('../data/fs/20170708_thai_fs.csv',parse_dates=[0],index_col=[0])
df.index=pd.DatetimeIndex(df.index)
df.index=df.index.date
df.sort_index(inplace=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
dfs

2000-03-31     100.078740
2000-06-30     100.125000
2000-09-30      99.872659
2000-12-31     100.038961
2001-03-31      99.788462
2001-06-30     100.126050
2001-09-30      99.977509
2001-12-31     100.047695
2002-03-31     100.031008
2002-06-30     100.006000
2002-09-30     100.042553
2002-12-31     184.408560
2003-03-31     399.368421
2003-06-30     401.163399
2003-09-30     400.280702
2003-12-31     400.294314
2004-03-31     400.776471
2004-06-30    2030.125000
2004-09-30    2019.372549
2004-12-31    2011.552239
2005-03-31    2027.333333
2005-06-30    2053.083333
2005-09-30    2095.438849
2005-12-31    2111.884615
2006-03-31    2176.095238
2006-06-30    2166.243902
2006-09-30    2163.672131
2006-12-31    2188.558442
2007-03-31    2188.263158
2007-06-30    2182.307692
2007-09-30    2177.931034
2007-12-31    2174.890244
2008-03-31    2204.928571
2008-06-30    2186.019231
2008-09-30    2191.358974
2008-12-31    2185.790000
2009-03-31    2154.214286
2009-06-30    2163.945455
2009-09-30  

In [18]:
#split_candidate
set50_tickers = ['PTT', 'SCC', 'AOT', 'ADVANC', 'SCB', 'KBANK', 'CPALL', 'BDMS',
       'BBL', 'KTB', 'PTTEP', 'PTTGC', 'CPN', 'INTUCH', 'TRUE', 'MINT',
       'BH', 'CPF', 'TOP', 'LH', 'BTS', 'GLOW', 'TMB', 'IVL', 'DELTA',
       'BLA', 'HMPRO', 'IRPC', 'TU', 'EGCO', 'SCCC', 'DTAC', 'TASCO',
       'BEC', 'CENTEL', 'PS', 'M', 'CK', 'SAWAD', 'BA', 'ROBINS', 'TCAP',
       'BCP', 'TPIPL', 'TTW', 'BANPU', 'WHA', 'ITD', 'CBG', 'JAS']
#loop over tickers
split_df=pd.DataFrame()
for ticker in set50_tickers:
    dfs=df[df.ticker==ticker]['number_of_shares']
    ratio = (dfs/dfs.shift(1)).apply(lambda x: round(x,1))
    ratio= ratio[ratio > 1.2].to_frame()
    ratio['ticker']=ticker
    split_df=split_df.append(ratio)
    
split_df.to_csv('/Users/Sunny/Desktop/split_candidate.csv')

In [21]:
df['cash_ttm']=df['cash'].rolling(window=4).mean()
df['market_cap']=df['close_price']*df['number_of_shares']

tmp=df.groupby('ticker')['eps'].apply(lambda x: 1.0*np.alltrue(x>0))
tmp.name='always_profit'
df=df.join(tmp,on='ticker')

tmp=df[(df.always_profit > 0)].groupby('ticker').last()[['cash_ttm','number_of_shares','close_price','market_cap']]
tmp['ppc']=tmp['close_price']*tmp['number_of_shares']/tmp['cash_ttm']
tmp[tmp > -float("inf")].sort_values(by='ppc')

,cash_ttm,number_of_shares,close_price,market_cap,ppc
ticker,,,,,
WG,49659.1750,17.839416,150.00,2675.912409,0.053886
PRAKIT,14433.1675,55.666667,15.10,840.566667,0.058239
UBIS,4218.4150,239.777778,5.80,1390.711111,0.329676
SENA,8341.8175,1178.333333,3.94,4642.633333,0.556549
CHG,49519.6225,15286.000000,2.50,38215.000000,0.771714
TOPP,777.1050,6.004992,190.00,1140.948419,1.468204
BM,774.6325,399.000000,3.92,1564.080000,2.019125
EKH,1486.3450,684.500000,6.45,4415.025000,2.970390
GIFT,638.0650,285.750000,7.70,2200.275000,3.448356


In [7]:
#filter

#good cashflow
df.groupby('ticker')['cash'].rolling(window=4).sum().reset_index()

,ticker,level_1,cash_flow_operation
0,2S,2009-06-30,NaN
1,2S,2009-09-30,NaN
2,2S,2009-12-31,NaN
3,2S,2010-03-31,NaN
4,2S,2010-06-30,40.32
5,2S,2010-09-30,-203.52
6,2S,2010-12-31,-29.34
7,2S,2011-03-31,160.20
8,2S,2011-06-30,-131.42
9,2S,2011-09-30,-52.94
